<a href="https://colab.research.google.com/github/davidguzmanr/CSC2516/blob/main/text-classifier/Membership-inference-BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/davidguzmanr/CSC2516.git

Cloning into 'CSC2516'...
remote: Enumerating objects: 190, done.
remote: Counting objects: 100% (190/190), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 190 (delta 114), reused 103 (delta 47), pack-reused 0
Receiving objects: 100% (190/190), 3.11 MiB | 4.82 MiB/s, done.
Resolving deltas: 100% (114/114), done.


In [2]:
%cd CSC2516/ 

/content/CSC2516


In [3]:
! pip install -r requirements.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.0/159.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.9/216.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 708.1/708.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.2/529.2 kB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 90.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 111.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import urllib.request
import zipfile
import os
from collections import OrderedDict
from tqdm.notebook import tqdm, trange

import numpy as np
import pandas as pd
import random
from numpy.random import choice
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision import models
from torchvision.datasets import CIFAR10
from torchvision import transforms

from transformers import BertConfig, BertForSequenceClassification, BertTokenizer
from datasets import load_dataset, DatasetDict, Dataset

from opacus.validators import ModuleValidator

from art.utils import load_mnist, load_cifar10
from art.estimators.classification.pytorch import PyTorchClassifier
from art.attacks.inference.membership_inference import MembershipInferenceBlackBoxRuleBased

SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)
random.seed(SEED)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Device: {device}')

Device: cuda:0


# Models

In [5]:
url = "https://csc2516-neural-networks-and-deep-learning.s3.amazonaws.com/text-classifier/lightning_logs.zip"
filename = "lightning_logs.zip"

urllib.request.urlretrieve(url, filename)

with zipfile.ZipFile('lightning_logs.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [6]:
def match_keys(state_dict: OrderedDict) -> OrderedDict:
    """
    Match the keys in a PyTorch Lightning state dictionary with a PyTorch format.
    
    Parameters:
    -----------
    state_dict : OrderedDict
        The state dictionary to be matched.
        
    Returns:
    --------
    OrderedDict
        A new state dictionary with the keys in a PyTorch format.
    """
    new_dict = OrderedDict()
    
    # For whatever reason PyTorch Lightning adds the word 'model.' at the beginning
    # of each key, so we have a mismatch in the keys. The problem goes away if we 
    # use our custom LightningModule to load the model, but I don't want to do that.
    for key in state_dict:
        # It saves the weights twice, in the second one it adds 'private_model.' to the key
        if 'private_model' not in key:
            # Remove the first part of the key, which is 'model.' for PyTorch Lightning
            new_key = '.'.join(key.split('.')[1:])
            new_dict[new_key] = state_dict[key]
        
    return new_dict

In [7]:
def find_pt_files(directory: str):
    """
    Recursively searches for files with the ".pt" extension in a given directory.
    Returns a list of file paths.

    Args:
        directory (str): The path to the directory to search for ".pt" files.

    Returns:
        A list of file paths to all the ".pt" files in the given directory and its subdirectories.
    """
    # Create an empty list to store the file paths of all ".pt" files found
    pt_files = []

    # Walk through the directory tree and search for files with the ".pt" extension
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".ckpt"):
                # If a file with the ".pt" extension is found, append its path to the list
                pt_files.append(os.path.join(root, file))
    
    # Return the list of file paths of all ".pt" files found
    return pt_files


In [8]:
def load_model(path: str, device: str):
    """
    Loads a PyTorch model from a saved checkpoint at the given path.
    The model is loaded onto the specified device.
    Returns the loaded model.

    Args:
        path (str): The path to the saved checkpoint.
        device (str): The device onto which the model should be loaded.
            Must be a string that PyTorch recognizes as a valid device identifier.

    Returns:
        The loaded PyTorch model.
    """
    # Load the checkpoint from the specified path
    checkpoint = torch.load(path, map_location=device)

    # Load BERT
    config = BertConfig.from_pretrained("bert-base-cased", num_labels=2)
    model = BertForSequenceClassification.from_pretrained("bert-base-cased", config=config)
    
    trainable_layers = [model.bert.encoder.layer[-1], model.bert.pooler, model.classifier]

    for p in model.parameters():
        p.requires_grad = False

    for layer in trainable_layers:
        for p in layer.parameters():
            p.requires_grad = True

    # Match the keys of the loaded state dict to the keys of the model's state dict
    state_dict = match_keys(checkpoint['state_dict'])

    # Load the state dict into the model, and move the model onto the specified device
    model.load_state_dict(state_dict)
    model.to(device)

    # Set the model to evaluation mode
    model.eval()

    # Return the loaded model
    return model

In [9]:
checkpoints = find_pt_files('lightning_logs/')
models_with_regularization = sorted([model for model in checkpoints if 'with-regularization' in model])
models_without_regularization = sorted([model for model in checkpoints if 'no-regularization' in model])

## Dataset

In [10]:
rotten_tomatoes = load_dataset('rotten_tomatoes')
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Dataset rotten_tomatoes downloaded and prepared to /root/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

## Attacks

In [11]:
class MembershipInferenceBlackBoxRuleBasedText:
    def __init__(self, model: BertForSequenceClassification, tokenizer: BertTokenizer) -> None:
        self.model = model
        self.tokenizer = tokenizer

    def tokenize_text(self, text: str) -> dict:
        """
        This function takes a text string and a tokenizer object from the Hugging Face
        Transformers library, and returns a dictionary of encoded inputs ready to be fed
        into a neural network.

        Parameters:
        text (str): the input text to be tokenized
        tokenizer (BertTokenizer): the tokenizer object to be used

        Returns:
        dict: a dictionary of encoded inputs (input_ids, token_type_ids, attention_mask)
        """
        
        # Tokenize the input text and create a dictionary of encoded inputs
        encoded_input = self.tokenizer(
            text,
            max_length=128,
            padding='max_length',
            truncation=True,
            return_tensors='pt',  # Use PyTorch tensors for the output
        )

        # Move the encoded inputs to the device used for training (e.g. GPU)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        encoded_input['input_ids'] = encoded_input['input_ids'].to(device)
        encoded_input['token_type_ids'] = encoded_input['token_type_ids'].to(device)
        encoded_input['attention_mask'] = encoded_input['attention_mask'].to(device)

        # Return the dictionary of encoded inputs
        return encoded_input

    def infer(self, dataset: Dataset) -> np.ndarray:
        """
        Runs inference on a dataset using the model and tokenizer provided in the constructor.
        Returns an array of binary values representing whether each input in the dataset was correctly
        classified by the model.

        Parameters:
        dataset (Dataset): The dataset to perform inference on

        Returns:
        np.ndarray: An array of binary values representing whether each input in the dataset was
            correctly classified by the model.
        """
        dataloader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=2)

        inferred = []
        for batch in tqdm(dataloader, leave=False):
            texts, labels = batch['text'], batch['label']
            encoded_inputs = self.tokenize_text(texts)
            preds = self.model(**encoded_inputs.to(device)).logits.argmax(dim=-1).cpu()
            inferred.extend([(label == pred).item() for (label, pred) in zip(labels, preds)])

        return np.array(inferred).astype(int)

    def predict(self, dataset: Dataset) -> np.ndarray:
        """
        Makes predictions on a dataset using the model and tokenizer provided in the constructor.
        Returns an array of predicted labels for each input in the dataset.

        Parameters:
        dataset (Dataset): The dataset to make predictions on

        Returns:
        np.ndarray: An array of predicted labels for each input in the dataset.
        """
        dataloader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=2)

        predictions = []
        for batch in tqdm(dataloader, leave=False):
            texts, labels = batch['text'], batch['label']
            encoded_inputs = self.tokenize_text(texts)
            preds = self.model(**encoded_inputs.to(device)).logits.argmax(dim=-1).cpu().tolist()
            predictions.extend(preds)

        return np.array(predictions)

In [16]:
def membership_inference_attack(model_path: str, tokenizer: BertTokenizer, dataset: DatasetDict, device: str):
    """
    Performs a membership inference attack on a PyTorch model.
    Returns the accuracy of the model on the test set, and the accuracy of the attack.

    Args:
        model_path (str): The path to the saved PyTorch model checkpoint.
        tokenizer (BertTokenizer): the tokenizer object to be used.
        dataset (DatasetDict): Dataset to evaluate.
        device (str): The device onto which the model should be loaded.
            Must be a string that PyTorch recognizes as a valid device identifier.

    Returns:
        A tuple containing the accuracy of the model on the test set, and the accuracy of the attack.
    """
    # Load the PyTorch model from the specified checkpoint
    model = load_model(model_path, device)

    # Set up the custom MembershipInferenceBlackBoxRuleBasedText attack
    attack = MembershipInferenceBlackBoxRuleBasedText(model, tokenizer)

    # Calculate accuracy in test
    predictions = attack.predict(dataset['validation'])
    y_test = np.array([sample['label'] for sample in dataset['validation']])
    accuracy = 100 * np.sum(predictions == y_test) / len(y_test)

    # Perform the attack on the training and test sets
    inferred_train = attack.infer(dataset['train'])
    inferred_test = attack.infer(dataset['validation'])

    # Calculate the accuracy of the attack
    train_acc = np.sum(inferred_train) / len(inferred_train)
    test_acc = 1 - (np.sum(inferred_test) / len(inferred_test))
    attack_acc = 100 * (train_acc * len(inferred_train) + test_acc * len(inferred_test)) / (len(inferred_train) + len(inferred_test))

    # Uncomment the following lines to print the accuracy results
    # print(f"Members Accuracy: {100 * train_acc:.4f}")
    # print(f"Non Members Accuracy {100 * test_acc:.4f}")
    # print(f"Attack Accuracy {100 * acc:.4f}")

    # Return the accuracy of the model and the accuracy of the attack as a tuple
    return accuracy, attack_acc


### Models with regularization

In [13]:
accuracies = []
attack_accuracies = []

for model in tqdm(models_with_regularization):
    accuracy, attack_accuracy = membership_inference_attack(
        model, tokenizer, rotten_tomatoes, device
    )

    accuracies.append(accuracy)
    attack_accuracies.append(attack_accuracy)


results_with_regularization = pd.DataFrame(
    data={
        'model': models_with_regularization,
        'test_accuracy': accuracies,
        'attack_accuracy': attack_accuracies,
    }
)

results_with_regularization['model'] = results_with_regularization['model'].apply(lambda name: name.split('/')[2])
# results_with_regularization['regularization'] = True
results_with_regularization.sort_values(by='attack_accuracy', ascending=False)

  0%|          | 0/8 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/134 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

...

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/134 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

,model,test_accuracy,attack_accuracy
0,bert,83.208255,78.418091
1,bert_data_augmentation,83.302064,77.469779
2,epsilon_10,73.545966,68.153397
3,epsilon_10_data_augmentation,72.795497,67.757399
6,epsilon_5,71.669794,66.715298
7,epsilon_5_data_augmentation,71.294559,66.475615
4,epsilon_3,71.013133,66.048353
5,epsilon_3_data_augmentation,70.825516,65.694039


### Models without regularization

In [14]:
accuracies = []
attack_accuracies = []

for model in tqdm(models_without_regularization):
    accuracy, attack_accuracy = membership_inference_attack(
        model, tokenizer, rotten_tomatoes, device
    )

    accuracies.append(accuracy)
    attack_accuracies.append(attack_accuracy)

results_without_regularization = pd.DataFrame(
    data={
        'model': models_without_regularization,
        'test_accuracy': accuracies,
        'attack_accuracy': attack_accuracies,
    }
)

results_without_regularization['model'] = results_without_regularization['model'].apply(lambda name: name.split('/')[2])
# results_without_regularization['regularization'] = False
results_without_regularization.sort_values(by='attack_accuracy', ascending=False)

  0%|          | 0/8 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/134 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

...

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/134 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

,model,test_accuracy,attack_accuracy
1,bert_data_augmentation,83.208255,79.793664
0,bert,82.176360,79.043351
2,epsilon_10,76.547842,70.466861
3,epsilon_10_data_augmentation,76.360225,70.112547
6,epsilon_5,74.015009,68.445185
7,epsilon_5_data_augmentation,73.733583,68.090871
4,epsilon_3,72.607880,67.476032
5,epsilon_3_data_augmentation,72.607880,67.007086


In [15]:
# results = pd.concat([results_with_regularization, results_without_regularization], ignore_index=True)
# results.sort_values(by='attack_accuracy', ascending=False)